Testing
=======

- Introduction
- Python `unittest` module
- Additional tools
- Continuous integration

# What is it?

- A task consisting of checking that the **program** is working as expected
- Manually written **tests** which can be automatically executed
- Different methodologies: Always and before anything else (test-driven development, *TDD*, [TDDwithPython])

<img src="img/ttd-workflow.png" style="width:40%;margin-left:auto;margin-right:auto;padding:2em;">

[TDDwithPython] `Harry J.W. Percival. Test-Driven Development with Python. O'Reilly 2014. [Oriented towards web development](http://chimera.labs.oreilly.com/books/1234000000754)

# Presenter Notes

- Test inject input to the program, and check output
- Answer valid or not
- For maintenance: Reproduce bug in a test, then fix it.


# Why testing?

- Validate the code to the specifications
- Find problems early
- Facilitates change
- Documentation
- Code quality

  - Better design
  - Simplifies integration


# Why not?

- Extra work
- Not perfect
- Extra maintenance

  - More difficult to refactoring
  - Maintain environments

- Delays integration

# Presenter Notes

- 30% percent of the time of the project
- Having the structure set-up for testing encourages writing tests.
- Then... let's talk about the structure :-)


# What kinds of tests?

- **Functional tests**: Tests scripts, public API, GUI
- **Integration tests**: Tests components, code correctly integrated
- **Unit tests**: Tests independant pieces of code

<img src="img/test-pyramid.png" style="width:40%;margin-left:auto;margin-right:auto;padding:0em;">

[TestPyramid] Mike Cohn. Succeeding with Agile: Software Development Using Scrum. 2009.


# Presenter Notes

The test pyramid is a concept developed by Mike Cohn, described in his book "Succeeding with Agile"

- Unit tests (dev point of view, fast, low cost)
- Integration tests
- Functional tests (user point of view, but slow, and expensive)

- Cost: unit << integration << functional
- Fast to execute: unit >> integration >> functional


# Where to put the tests?

Separate tests from the source code:

- Run the test from the command line.
- Separate test code when distributing.
- [...](https://docs.python.org/3/library/unittest.html#organizing-test-code)

Folder structure:

- In a separate `test/` folder.
- In `test` sub-packages in each Python package/sub-package,
  so that tests remain close to the source code.
  Tests are installed with the package and can be run from the installation.
- A `test_*.py` for each module and script (an more if needed).
- Consider separating tests that are long to run from the others.


# Where to put the tests?

- project
  - setup.py
  - run_tests.py
  - package/
    - __init__.py
    - module1.py
    - test/
      - __init__.py
      - test_module1.py
    - subpackage/
      - __init__.py
      - module1.py
      - module2.py
      - test/
        - __init__.py
        - test_module1.py
        - test_module2.py

# Presenter Notes

- scripts/
  - my_script.py
  - my_other_script.py
- test/
  - test_my_script.py
  - test_my_other_script.py

# `unittest` Python module

- QuickStart
- Chaining tests
- Running tests
- More on TestCase


# QuickStart

[unittest](https://docs.python.org/3/library/unittest.html) is the default Python module for testing.

It provides features to:

- Write tests
- Discover tests
- Run those tests


# TestCase

The classe `unittest.TestCase` is the base class for writting tests for
Python code.


In [96]:
import unittest

class TestMyTestCase(unittest.TestCase):

    def test_my_test(self):
        # Test code
        self.assertEqual(a, b, message=None)

    def test_another_test(self):
        self.assertTrue(x, message=None)

# Assertion functions

- Argument(s) to compare/evaluate.
- An additional error message.

- `assertEqual(a, b)` checks that `a == b`
- `assertNotEqual(a, b)` checks that `a != b`
- `assertTrue(x)` checks that `bool(x) is True`
- `assertFalse(x)`checks that `bool(x) is False`
- `assertIs(a, b)` checks that `a is b`
- `assertIsNone(x)` checks that `x is None`
- `assertIn(a, b)` checks that `a in b`
- `assertIsInstance(a, b)` checks that `isinstance(a, b)`

There's more, see [unittest TestCase documentation](https://docs.python.org/3/library/unittest.html#unittest.TestCase>)
or [Numpy testing documentation](http://docs.scipy.org/doc/numpy/reference/routines.testing.html).

# Run the tests

In [ ]:
import unittest

if __name__ == "__main__":
    unittest.main()

The function `unittest.main()` provides a command line interface to
discover and run the tests.

# Example

In [ ]:
import unittest

class TestBuiltInRound(unittest.TestCase):

    def test_positive(self):
        result = round(1.3)
        self.assertEqual(result, 1)

    def test_negative(self):
        result = round(-1.3)
        self.assertEqual(result, -1)

    def test_halfway_even(self):
        result = round(2.5)
        self.assertEqual(result, 2, msg="round(2.5) -> %f != 2" % result)

    def test_returned_type(self):
        self.assertIsInstance(round(0.), int)

if __name__ == "__main__":
    unittest.main()

# Example: Result in Python3

Running tests from the command line on Python3:

# Example: Make it fail

# Example: Command line arguments

Running a specific `TestCase`:

Running a specific test method:

# Chaining tests

How-to run tests from many ``TestCase`` and many files at once:

- Explicit:
    Full control, boilerplate code.

- Automatic:
    No control

- Mixing approach

# Chaining tests: Suite

The [TestSuite](https://docs.python.org/3/library/unittest.html#unittest.TestSuite) class aggregates test cases and test suites through:

- `TestSuite.addTest(test)`
- `TestSuite.addTests(test)`

Example:

In [ ]:
suite = unittest.TestSuite()
suite.addTest(TestBuiltInRound('test_positive'))
...
...
...


# Chaining tests: Loader

`unittest.defaultTestLoader` (an instance of `unittest.TestLoader`) creates `TestSuite` from classes and modules.

`TestLoader.loadTestsFromTestCase(testCaseClass)`` method creates a `TestSuite` from all `test*` method of a `TestCase` subclass.

In [79]:
loadTests = unittest.defaultTestLoader.loadTestsFromTestCase
suite = unittest.TestSuite()
suite.addTest(loadTests(TestBuiltInRound))

# Chaining tests: Module

First, write a ``suite`` function for each module (i.e., file):

In [81]:
def suite():
    loadTests = unittest.defaultTestLoader.loadTestsFromTestCase
    suite = unittest.TestSuite()
    suite.addTest(loadTests(TestBuiltInRound))
    return suite

# Chaining tests: Package

Then a ``suite`` function collecting all tests in a package (i.e., directory).

In [ ]:
from . import test_builtin_round
...

def suite():
    suite = unittest.TestSuite()
    suite.addTest(test_builtin_round.suite())
    ...
    return suite

This can be used to create a `TestSuite` from all tests in a project:

- Full control over the creation of the `TestSuite`.
- Requires some boilerplate code.

# Chaining tests: Runner

To run the ``suite`` from command line:


In [ ]:
def suite():
    ...

if __name__ == "__main__":  # True if run as a script
    unittest.main(defaultTest='suite')


# Project: Running tests

- `unittest.main` to run each module independantly.
- Command line: `python -m unittest ...`
- With a `run_tests.py` script.

Minimal run_tests.py:


In [ ]:
import unittest
import mymodule.tests

runner = unittest.TextTestRunner()
runner.run(mymodule.tests.suite())

# Sum-up

- For each modules
    - Write a test module with tests as `TestCase` sub-class
    - Use `assert` methods in the tests
    - Run the tests as a script from the command line

- For packages and project
    - Chain tests with `TestSuite`
    - Create a script to run your tests

# More features

- Fixture
- Testing exception
- Skipping tests
- Parametric tests
- Test data


# Fixture

Tests might need to share some common initialisation/finalisation (e.g., create a temporary directory).

This can be implemented in ``setUp`` and ``tearDown`` methods of ``TestCase``.
Those methods are called before and after each test.

In [88]:
class TestCaseWithFixture(unittest.TestCase):

    def setUp(self):
        pass  # Pre-test code

    def tearDown(self):
        pass  # Post-test code

    pass  # Tests

# Module fixture


In [89]:
def setUpModule():
    # Called before all the tests of this module
    pass

def tearDownModule():
    # Called after all the tests of this module
    pass

# Class fixture

In [90]:
class TestSample(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        # Called before all the tests of this class
        pass

    @classmethod
    def tearDownClass(cls):
        # Called after all the tests of this class
        pass

# Testing exception

In [97]:
class TestBuiltInRound(unittest.TestCase):

    def test_raise_type_error(self):
        with self.assertRaises(TypeError):
            result = round('2')

`TestCase.assertRaisesRegexp` also checks the message of the exception.

# Skipping tests

Why skipping a test: Test requires a specific OS or a specific version of a library...

To skip a test, call `TestCase.skipTest(reason)` from the test* or `setUp` method.

Also available through decorators `unittest.skip`, `unittest.skipIf`, `unittest.skipUnless`.


In [92]:
import sys
import unittest

class TestBuiltInRound(unittest.TestCase):

    def test_python2(self):
        if sys.version_info[0] != 2:
            self.skipTest('Requires Python 2')
        self.assertEqual(round(2.5), 3.0)

    @unittest.skipIf(sys.version_info[0] != 3, 'Requires Python 3')
    def test_python3(self):
        self.assertEqual(round(2.5), 2)

# Parametric tests

Running the same test with multiple values

Problems:

- The first failure stops the test, remaining test values are not processed.
- There is no information on the value for which the test has failed.

In [100]:
class TestBuiltInRound(unittest.TestCase):

    HALFWAY_TESTS = ((0.5, 0), (1.5, 2), (2.5, 2))

    def test_all(self):
        for value, expected in self.HALFWAY_TESTS:
            self.assertEqual(round(value), expected)

    def test_all_better_way(self):
        for value, expected in self.HALFWAY_TESTS:
            with self.subTest(value=value, expected=expected):
                self.assertEqual(round(value), expected)

# Test data

How to handle test data?

Need to separate (possibly huge) test data from python package.

Download test data and store it in a temporary directory during the tests if not available.

Example: [silx.utils.ExternalResources](https://github.com/silx-kit/silx/blob/master/silx/utils/utilstest.py)

# QTest

For GUI based on `PyQt`, `PySide` it is possible to use Qt's [QTest](http://doc.qt.io/qt-5/qtest.html).

It provides the basic functionalities for GUI testing.
It allows to send keyboard and mouse events to widgets.

In [66]:
from PyQt5 import Qt

qapp = Qt.QApplication.instance()
if Qt.QApplication.instance() is None:
    qapp = Qt.QApplication([])

In [65]:
from PyQt5 import Qt

class MyApplication(Qt.QMainWindow):

    def __init__(self, parent=None):
        super(MyApplication, self).__init__(parent=parent)
        self.initGui()

    def initGui(self):
        self._inputLine = Qt.QLineEdit(self)
        self._processButton = Qt.QPushButton(self)
        self._processButton.setText("Process")
        self._processButton.clicked.connect(self.processing)
        self._resultWidget = Qt.QLabel(self)

        widget = Qt.QWidget()
        layout = Qt.QVBoxLayout(widget)
        layout.addWidget(self._inputLine)
        layout.addWidget(self._resultWidget)
        layout.addWidget(self._processButton)
        self.setCentralWidget(widget)

    def processing(self):
        # Compute the sum of values displayed in the line edit
        text = self._inputLine.text()
        data = [int(d) for d in text.split()]
        self.result = sum(data)
        self._resultWidget.setText("%d" % self.result)

In [102]:
from PyQt5 import Qt
from PyQt5.QtTest import QTest

app = MyApplication()
app.show()
QTest.qWaitForWindowExposed(widget)
QTest.keyClicks(app._inputLine, '1 10 100', delay=100)  # Wait 100ms
QTest.mouseClick(app._processButton, QtCore.Qt.LeftButton, pos=QtCore.QPoint(1, 1))
assert app._resultWidget.text() == "111"

Tighly coupled with the code it tests.
It needs to know the widget's instance and hard coded position of mouse events.

# Test coverage

Using [`coverage`](https://coverage.readthedocs.org) to gather coverage statistics while running the tests:

- Install `coverage.py` package: `pip install coverage`.
- Run the tests: `python -m coverage run --source <package_dir> run_tests.py`
- Show report:

  - `python -m coverage report`
  - `python -m coverage html`

# Extra test tools

Extending `unittest`:

- [pytest](http://pytest.org/)
- [nose](https://nose.readthedocs.org/)

Running the tests on different Python environments:

- [tox](https://tox.readthedocs.org/) - automates testing of Python packages


# Continuous integration

Benefits:

- Test on multiple platform/configuration (e.g., different version of Python).
- Test often: each commit, each pull request, daily...

Costs:

- Set-up and maintenance.
- Test needs to be automated.


# Continuous integration

- [Travis-CI](https://travis-ci.org/): Linux and MacOS
- [AppVeyor](http://www.appveyor.com/): Windows
- gitlab-CI (https://gitlab.esrf.fr)
- ...

Principle:

- Add a `.yml` file to your repository describing:

  - The test environment
  - Build and installation of the dependencies and the package
  - The way to run the tests.

- Upon commit, clones the repository and runs the tests.
- Displays the outcome on a web page.
- Feedback github Pull Requests with test status.


# Continuous integration: Configuration

Example of configuration with Travis

# Sum-up

- Different test strategies.
- Python `unittest` (and extra packages) to write and run the tests.
- Additional tools to efficiently run the tests: Continuous Integration.
- Next step: Continuous Deployment.